In [5]:
## import all dependencies

import numpy as np
import pandas as pd
import _pickle as cPickle
from collections import defaultdict
import re
import sys
import os

os.environ['KERAS_BACKEND']='theano'

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical

from keras.layers import Embedding
from keras.layers import Dense, Input, Flatten
from keras.layers import Conv1D, MaxPooling1D, Embedding, merge, Dropout
from keras.models import Model

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using Theano backend.
WARNING (theano.configdefaults): g++ not available, if using conda: `conda install m2w64-toolchain`
C:\ProgramData\Anaconda3\lib\site-packages\theano\configdefaults.py:560: UserWarning: DeprecationWarning: there is no c++ compiler.This is deprecated and with Theano 0.11 a c++ compiler will be mandatory
  warnings.warn("DeprecationWarning: there is no c++ compiler."
WARNING (theano.configdefaults): g++ not detected ! Theano will be unable to execute optimized C-implementations (for both CPU and GPU) and will default to Python implementations. Performance will be severely degraded. To remove this warning, set Theano flags cxx to an empty string.
WARNING (theano.te

In [6]:
## param list

## parameter list

MAX_SEQUENCE_LENGTH = 300
MAX_NB_WORDS = 20000
EMBEDDING_DIM = 300
DROPOUT_RATE    = 0.4
HIDDEN_UNITS    = 200

# NB_CLASSES      = 2

# LEARNING
# BATCH_SIZE      = 100
# NB_EPOCHS       = 10
# RUNS            = 5
# VAL_SIZE        = 0.2

VALIDATION_SPLIT = 0.2

In [7]:
## data preprocessing
import pandas as pd
from nltk.corpus import stopwords
import string,re

dataset=pd.read_csv('train.tsv',delimiter='\t',encoding='utf-8')
dataset.columns=['ID','Label','Statement','Subject','speaker','job_title',
           'state_info','pantry_affiliation','barely_true_cnt','false_cnt',
           'half_true_cnt','mostly_true_cnt','pants_on_fire_cnt','Context']

def preprocessing_txt(dataset):
    stop_words = set(stopwords.words('english'))
    rm_punct = re.compile('[{}]'.format(re.escape(string.punctuation)))
    corpus=[]
    for elm in range(0, len(dataset.index)):
        res=' '.join([i for i in dataset['Statement'][elm].split() if i not in stop_words])
        res=rm_punct.sub(' ', res)
        corpus.append(res)
    return corpus

corpus=preprocessing_txt(dataset)

In [8]:
## import local dependencies
import util
from nlp_util import NLP_Task
import preprocessing
import importlib
importlib.reload(preprocessing)


## label
multi_labels = {'false':0, 'true':1,'pants-fire':2,'barely-true':3,'half-true':4,'mostly-true':5}
binary_labels = {'false':1, 'true':-1,'pants-fire':1,'barely-true':1,'half-true':0,'mostly-true':-1}

dataset['b_label'] = np.array(preprocessing.create_labels(labels=dataset['Label'].values,label_values=binary_labels))
dataset['m_label'] = np.array(preprocessing.create_labels(labels=dataset['Label'].values,label_values=multi_labels))

texts = corpus
labels = dataset['b_label']

labels = to_categorical(np.asarray(labels))
print('Shape of data tensor:', dataset.shape)
print('Shape of label tensor:', labels.shape)

#
# indices = np.arange(dataset.shape[0])
# np.random.shuffle(indices)


C:\ProgramData\Anaconda3\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
C:\ProgramData\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: cymem.cymem.Pool size changed, may indicate binary incompatibility. Expected 48 from C header, got 64 from PyObject
  return f(*args, **kwds)
C:\ProgramData\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: cymem.cymem.Address size changed, may indicate binary incompatibility. Expected 24 from C header, got 40 from PyObject
  return f(*args, **kwds)


NLP_Task ready to use.
NLP_Task ready to use.
Shape of data tensor: (10239, 16)
Shape of label tensor: (10239, 2)


In [9]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

# cv = CountVectorizer()
# corpus = cv.fit_transform(corpus).toarray()

X_train, X_val, y_train, y_val  = train_test_split(corpus,
                                                   labels,train_size=0.8, random_state=123)


print('Number of positive and negative reviews in traing and validation set ')
print(y_train.sum(axis=0))
print(y_val.sum(axis=0))

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


Number of positive and negative reviews in traing and validation set 
[1673. 6518.]
[ 441. 1607.]


In [10]:
##
t = Tokenizer(num_words=5000) # create tokenizer with a max number of words to take into account according to frequency
t.fit_on_texts(X_train) # fit tokenizer with train data
X_train = t.texts_to_sequences(X_train)
X_dev = t.texts_to_sequences(X_val)
X_train = pad_sequences(X_train,maxlen=MAX_SEQUENCE_LENGTH,padding='post')
X_dev = pad_sequences(X_dev, maxlen=MAX_SEQUENCE_LENGTH, padding='post')

In [11]:
import numpy as np
def loadGloveModel(gloveFile):
    print("Loading Glove Model")
    embeddings_index = {}
    f = open(gloveFile, encoding='utf8')
    for line in f:
        values = line.split()
        word = ''.join(values[:-300])
        coefs = np.asarray(values[-300:], dtype='float32')
        embeddings_index[word] = coefs
    f.close()
    return embeddings_index

In [12]:

glove_model = loadGloveModel('glove.6B.300d.txt')

Loading Glove Model


In [13]:
glove_model['computer']

array([-2.7628e-01,  1.3999e-01,  9.8519e-02, -6.4019e-01,  3.1988e-02,
        1.0066e-01, -1.8673e-01, -3.7129e-01,  5.9740e-01, -2.0405e+00,
        2.2368e-01, -2.6314e-02,  7.2408e-01, -4.3829e-01,  4.8886e-01,
       -3.5486e-03, -1.0006e-01, -3.0587e-01, -1.5621e-01, -6.8136e-02,
        2.1104e-01,  2.9287e-01, -8.8861e-02, -2.0462e-01, -5.7602e-01,
        3.4526e-01,  4.1390e-01,  1.7917e-01,  2.5143e-01, -2.2678e-01,
       -1.0103e-01,  1.4576e-01,  2.0127e-01,  3.1810e-01, -7.8907e-01,
       -2.2194e-01, -2.4833e-01, -1.5103e-02, -2.0050e-01, -2.6441e-02,
        1.8551e-01,  3.3782e-01, -3.3543e-01,  8.6117e-01, -4.7083e-02,
       -1.7009e-01,  3.0438e-01,  9.4119e-02,  3.2435e-01, -8.1171e-01,
        8.8966e-01, -3.9149e-01,  1.6828e-01,  1.4316e-01,  3.6339e-03,
       -6.4557e-02,  4.5777e-02, -3.2248e-01,  4.8943e-02,  1.6817e-01,
        6.8344e-02,  5.4227e-01,  1.2493e-01,  6.9742e-01, -3.7194e-02,
        3.3080e-01, -4.2194e-01,  3.3970e-01,  2.7646e-01, -1.60

In [14]:
## build glove-vec embedding layer
def build_glove_embedding_layers():
    """"embed_index={}
    with open('glove.840B.300d.txt'.format(EMBEDDING_DIM), encoding="utf8",) as file:
        for line in file:
            values=line.split()
            word=values[0]
            coefs=np.asarray(values[1:], dtype='float32')
            embed_index[word]=coefs"""
    embed_matrix=np.zeros((MAX_NB_WORDS, EMBEDDING_DIM))
    for word, indx in t.word_index.items():
        if indx >= MAX_NB_WORDS:
            continue
        if word in glove_model:
            embed_vec=glove_model[word]
            if embed_vec is not None:
                embed_matrix[indx]=embed_vec
    embed_matrix
    return Embedding(input_dim=MAX_NB_WORDS,
                     output_dim=EMBEDDING_DIM,
                     input_length=MAX_SEQUENCE_LENGTH,
                     weights=[embed_matrix],
                     trainable=False, name='word embedding')

##
embedding_layers=build_glove_embedding_layers()

In [15]:
embedding_layers

In [16]:
from keras.layers import concatenate
#concatenate([x1, x2], axis=-1)

In [17]:
# applying complex convolutional approach
convs = []
filter_sizes = [3,4,5]

sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layers(sequence_input)

for fsz in filter_sizes:
    l_conv = Conv1D(nb_filter=128,filter_length=fsz,activation='relu')(embedded_sequences)
    l_pool = MaxPooling1D(5)(l_conv)
    convs.append(l_pool)
    

    
l_merge = concatenate(convs, axis=1)
l_cov1= Conv1D(128, 5, activation='relu')(l_merge)
l_pool1 = MaxPooling1D(5)(l_cov1)
l_cov2 = Conv1D(128, 5, activation='relu')(l_pool1)
l_pool2 = MaxPooling1D(30)(l_cov2)
l_flat = Flatten()(l_pool2)
l_flat = Dense(HIDDEN_UNITS)(l_flat)
l_flat = Dropout(DROPOUT_RATE)(l_flat)
l_dense = Dense(128, activation='relu')(l_flat)

preds = Dense(2, activation='softmax')(l_dense)

model = Model(sequence_input, preds)
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])

print("model fitting - more complex convolutional neural network")
model.summary()
model.fit(X_train, y_train, validation_data=(X_dev, y_val),
          nb_epoch=3, batch_size=50)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(activation="relu", filters=128, kernel_size=3)`
  if __name__ == '__main__':
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(activation="relu", filters=128, kernel_size=4)`
  if __name__ == '__main__':
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(activation="relu", filters=128, kernel_size=5)`
  if __name__ == '__main__':


model fitting - more complex convolutional neural network
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 300)          0                                            
__________________________________________________________________________________________________
word embedding (Embedding)      (None, 300, 300)     6000000     input_1[0][0]                    
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 298, 128)     115328      word embedding[0][0]             
__________________________________________________________________________________________________
conv1d_2 (Conv1D)               (None, 297, 128)     153728      word embedding[0][0]             
___________________________________________________

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:33: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
C:\ProgramData\Anaconda3\lib\site-packages\theano\tensor\nnet\conv.py:98: UserWarning: theano.tensor.nnet.conv.conv2d is deprecated. Use theano.tensor.nnet.conv2d instead.
  warnings.warn("theano.tensor.nnet.conv.conv2d is deprecated."


Train on 8191 samples, validate on 2048 samples
Epoch 1/3


C:\ProgramData\Anaconda3\lib\site-packages\theano\tensor\signal\pool.py:574: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  yk[[region_slices[i][r[i]] for i in xrange(nd)]])
C:\ProgramData\Anaconda3\lib\site-packages\theano\tensor\signal\pool.py:574: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  yk[[region_slices[i][r[i]] for i in xrange(nd)]])
C:\ProgramData\Anaconda3\lib\site-packages\theano\tensor\signal\pool.py:574: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `

...

 550/8191 [=>............................] - ETA: 33:39:34 - loss: 0.6687 - acc: 0.7600

C:\ProgramData\Anaconda3\lib\site-packages\theano\tensor\signal\pool.py:574: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  yk[[region_slices[i][r[i]] for i in xrange(nd)]])
C:\ProgramData\Anaconda3\lib\site-packages\theano\tensor\signal\pool.py:574: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  yk[[region_slices[i][r[i]] for i in xrange(nd)]])
C:\ProgramData\Anaconda3\lib\site-packages\theano\tensor\signal\pool.py:574: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `

 600/8191 [=>............................] - ETA: 42:55:41 - loss: 0.6678 - acc: 0.7533

C:\ProgramData\Anaconda3\lib\site-packages\theano\tensor\signal\pool.py:574: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  yk[[region_slices[i][r[i]] for i in xrange(nd)]])
C:\ProgramData\Anaconda3\lib\site-packages\theano\tensor\signal\pool.py:574: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  yk[[region_slices[i][r[i]] for i in xrange(nd)]])
C:\ProgramData\Anaconda3\lib\site-packages\theano\tensor\signal\pool.py:574: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `

SystemError: <class 'range'> returned a result with an error set
Apply node that caused the error: Elemwise{Composite{((i0 * i1) + (i0 * i1 * sgn(i2)))}}[(0, 1)](TensorConstant{(1, 1, 1) of 0.5}, InplaceDimShuffle{0,2,1}.0, Elemwise{Add}[(0, 0)].0)
Toposort index: 265
Inputs types: [TensorType(float32, (True, True, True)), TensorType(float32, 3D), TensorType(float32, 3D)]
Inputs shapes: [(1, 1, 1), (50, 298, 128), (50, 298, 128)]
Inputs strides: [(4, 4, 4), (152576, 4, 1192), (152576, 4, 1192)]
Inputs values: [array([[[0.5]]], dtype=float32), 'not shown', 'not shown']
Outputs clients: [[Sum{axis=[0, 1], acc_dtype=float64}(Elemwise{Composite{((i0 * i1) + (i0 * i1 * sgn(i2)))}}[(0, 1)].0), Reshape{3}(Elemwise{Composite{((i0 * i1) + (i0 * i1 * sgn(i2)))}}[(0, 1)].0, Join.0)]]

HINT: Re-running with most Theano optimization disabled could give you a back-trace of when this node was created. This can be done with by setting the Theano flag 'optimizer=fast_compile'. If that does not work, Th

In [ ]:
import pickle

In [ ]:
pickle.dump(model, open('CNN_model', 'wb'))